In [1]:
import pandas as pd, numpy as np, sqlite3 as sql
import websocket, talib, json, telegram, time
from collections import deque
from datetime import datetime, timedelta
from binance.client import Client
from config import *

In [2]:
con = sql.connect('kubera.db')

In [3]:
def macd_sar_signals(data):
    data = data[:4]
    macd_cross_up, macd_cross_down, macd_below_zero, macd_above_zero = [False] * 4
    sar_shift_up, sar_shift_down, sar_below_low, sar_above_high = [False] * 4
        
    for i in range(len(data)-1):
        if data[i]['macdhist'] > 0 and data[i+1]['macdhist'] < 0:
            macd_cross_up = True
            break
        elif data[i]['macdhist'] < 0 and data[i+1]['macdhist'] > 0:
            macd_cross_down = True
            break

    if data[0]['macd'] < 0 or data[-1]['macd'] < 0:
        macd_below_zero = True
    elif data[0]['macd'] > 0 or data[-1]['macd'] > 0:
        macd_above_zero = True

    if data[0]['sar'] > data[0]['high']:
        sar_above_high = True
    elif data[0]['sar'] < data[0]['low']:
        sar_below_low = True
        
    for i in range(len(data)-1):
        if data[i]['sar'] < data[i]['low'] and data[i+1]['sar'] > data[i+1]['high']:
            sar_shift_down = True
            break
        elif data[i]['sar'] > data[i]['high'] and data[i+1]['sar'] < data[i+1]['low']:
            sar_shift_up = True
            break
    
    aux_data = {'macd_cross_up':macd_cross_up, 'macd_cross_down':macd_cross_down, \
                'macd_below_zero':macd_below_zero, 'macd_above_zero':macd_above_zero, \
                'sar_shift_up':sar_shift_up, 'sar_shift_down':sar_shift_down, \
                'sar_below_low':sar_below_low, 'sar_above_high':sar_above_high}
    
    if macd_cross_up and sar_shift_down and macd_below_zero:
        return 'b', aux_data
    elif macd_cross_down or sar_above_high:
        return 'so', aux_data
    elif macd_cross_down and sar_shift_up and macd_above_zero:
        return 's', aux_data
    elif macd_cross_up or sar_below_low:
        return 'bo', aux_data
    else: return 'h', aux_data

In [4]:
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'n_trades', \
           'macd', 'macdsignal', 'macdhist', 'rsi_slowk', 'rsi_slowd', 'obv', 'atr', 'ema200', 'sar']

def get_new_data(con):
    cur = con.cursor()
    data = []
    for row in cur.execute('SELECT * FROM stream ORDER BY timestamp DESC LIMIT 4'):
        data.append(dict(zip(columns, row)))
    return data

In [ ]:
time_dict = {'1m':60, '3m':60*3, '5m':60*5, '15m':60*15, '30m':60*30, '1h':60*60*1, \
             '2h':60*60*2, '4h':60*60*4, '6h':60*60*6, '8h':60*60*8, '12h':60*60*12, \
             '1d':60*60*24*1, '3d':60*60*24*3, '1w':60*60*24*7, '1M':60*60*24*30}

proccessed_timestamps = deque([], maxlen=1000)

investment = 300
open_long_position = False
open_short_position = False
open_long_price = 0
open_short_price = 0
number_of_trades = 0 

for i in range(10000):
    current_time = datetime.utcnow()
    current_time = (datetime(current_time.year, current_time.month, current_time.day, current_time.hour, current_time.minute) + timedelta(hours=1)).timestamp()
    data = get_new_data(con)
    if data[0]['timestamp'] - current_time <= time_dict['5m']*1.25 and data[0]['timestamp'] not in proccessed_timestamps:
        signal, aux = macd_sar_signals(data)
        formatted_time = datetime.fromtimestamp(current_time).strftime("%b %d %H:%M")
        price = data[0]['close']
        if signal == 'b':
            if not open_long_position and not open_short_position:
                print(f"\n{formatted_time} ({signal}) : lllll BUY lllll high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) \n")
                open_long_position = True
                open_long_price = price
                number_of_trades += 1
            else: print(f"{formatted_time} ({signal}) : HOLD high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) ")
        elif signal == 'so':
            if open_long_position:
                print(f"\n{formatted_time} ({signal}) : lllll SELL lllll high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) \n")
                print(f"\nProfit made: {(price - open_long_price) * investment} Wallet Balance: {investment + (price - open_long_price) * investment}, Number of trades: {number_of_trades+1}\n")
                investment += (price - open_long_price) * investment
                open_long_position = False
                number_of_trades += 1
            else: print(f"{formatted_time} ({signal}) : HOLD high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) ")
        elif signal == 's':
            if not open_long_position and not open_short_position:
                print(f"\n{formatted_time} ({signal}) : sssss SELL sssss high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) \n")
                open_short_position = True
                open_short_price = price
                number_of_trades += 1
            else: print(f"{formatted_time} ({signal}) : HOLD high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) ")
        elif signal == 'bo':
            if open_short_position:
                print(f"\n{formatted_time} ({signal}) : sssss BUY sssss high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) \n")
                print(f"\nProfit made: {(open_short_price - price) * investment} Wallet Balance: {investment + (open_short_price - price) * investment}, Number of trades: {number_of_trades+1}\n")
                investment += (open_short_price - price) * investment
                open_short_position = False
                number_of_trades += 1
            else: print(f"{formatted_time} ({signal}) : HOLD high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) ")
        else:
            print(f"{formatted_time} ({signal}) : HOLD high {data[0]['high']}, low {data[0]['low']}, sar {round(data[0]['sar'], 4)}, shift_up({aux['sar_shift_up']}) shift_down({aux['sar_shift_down']}) below_low({aux['sar_below_low']}) above_high({aux['sar_above_high']}), macd {round(data[0]['macd'], 4)}, macdhist {round(data[0]['macdhist'], 4)}, cross_up({aux['macd_cross_up']}) cross_down({aux['macd_cross_down']}) below_zero({aux['macd_below_zero']}) above_zero({aux['macd_above_zero']}) ")
        proccessed_timestamps.append(data[0]['timestamp'])
    time.sleep(time_dict['5m']/3)

Aug 18 21:21 (bo) : HOLD high 0.80822, low 0.80477, sar 0.7952, shift_up(False) shift_down(True) below_low(True) above_high(False), macd -0.005, macdhist -0.0005, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 18 21:26 (bo) : HOLD high 0.8092, low 0.8028, sar 0.7958, shift_up(False) shift_down(False) below_low(True) above_high(False), macd -0.0044, macdhist 0.0, cross_up(True) cross_down(False) below_zero(True) above_zero(False) 
Aug 18 21:31 (bo) : HOLD high 0.81297, low 0.80785, sar 0.7964, shift_up(False) shift_down(False) below_low(True) above_high(False), macd -0.0037, macdhist 0.0006, cross_up(True) cross_down(False) below_zero(True) above_zero(False) 
Aug 18 21:36 (bo) : HOLD high 0.81468, low 0.80901, sar 0.7974, shift_up(False) shift_down(False) below_low(True) above_high(False), macd -0.003, macdhist 0.001, cross_up(True) cross_down(False) below_zero(True) above_zero(False) 
Aug 18 21:41 (bo) : HOLD high 0.81388, low 0.80675, sar 0.7988, shift_up(Fa

Aug 19 00:21 (bo) : HOLD high 0.83378, low 0.83132, sar 0.8213, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0064, macdhist 0.0018, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 00:26 (bo) : HOLD high 0.83789, low 0.834, sar 0.8237, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0069, macdhist 0.0018, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 00:31 (bo) : HOLD high 0.83561, low 0.83293, sar 0.8258, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.007, macdhist 0.0016, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 00:36 (bo) : HOLD high 0.847, low 0.83461, sar 0.8276, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0076, macdhist 0.0018, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 00:41 (bo) : HOLD high 0.84547, low 0.83498, sar 0.8307, shift_up(

Aug 19 07:46 (bo) : HOLD high 0.80442, low 0.80194, sar 0.7973, shift_up(False) shift_down(True) below_low(True) above_high(False), macd -0.0012, macdhist -0.0001, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 07:51 (so) : HOLD high 0.80083, low 0.79744, sar 0.8044, shift_up(True) shift_down(False) below_low(False) above_high(True), macd -0.0014, macdhist -0.0002, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 

Aug 19 07:56 (b) : lllll BUY lllll high 0.80699, low 0.79758, sar 0.7974, shift_up(False) shift_down(True) below_low(True) above_high(False), macd -0.001, macdhist 0.0002, cross_up(True) cross_down(False) below_zero(True) above_zero(False) 

Aug 19 08:01 (b) : HOLD high 0.80594, low 0.80483, sar 0.7974, shift_up(False) shift_down(True) below_low(True) above_high(False), macd -0.0006, macdhist 0.0004, cross_up(True) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 08:06 (b) : HOLD high 0.81034, low 0.80411, sar 0.7

Aug 19 11:01 (bo) : HOLD high 0.81567, low 0.81387, sar 0.8054, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0015, macdhist 0.0002, cross_up(True) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 11:06 (bo) : HOLD high 0.81518, low 0.81321, sar 0.8059, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0015, macdhist 0.0002, cross_up(True) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 11:11 (bo) : HOLD high 0.81503, low 0.81175, sar 0.8063, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0014, macdhist 0.0, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 11:16 (so) : HOLD high 0.81226, low 0.80823, sar 0.8067, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.001, macdhist -0.0003, cross_up(False) cross_down(True) below_zero(False) above_zero(True) 
Aug 19 11:21 (so) : HOLD high 0.81008, low 0.80641, sar 0.8165, shift_up(T

Aug 19 13:56 (bo) : HOLD high 0.80338, low 0.8021, sar 0.7908, shift_up(False) shift_down(False) below_low(True) above_high(False), macd -0.0006, macdhist 0.0011, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 14:01 (bo) : HOLD high 0.80835, low 0.8037, sar 0.7918, shift_up(False) shift_down(False) below_low(True) above_high(False), macd -0.0002, macdhist 0.0012, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 14:06 (bo) : HOLD high 0.80701, low 0.8034, sar 0.7932, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0002, macdhist 0.0013, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 14:11 (bo) : HOLD high 0.80701, low 0.805, sar 0.7944, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0004, macdhist 0.0012, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 19 14:16 (bo) : HOLD high 0.80682, low 0.80522, sar 0.7955, shift_u

Aug 19 16:51 (bo) : HOLD high 0.82641, low 0.8219, sar 0.8121, shift_up(False) shift_down(True) below_low(True) above_high(False), macd 0.0029, macdhist 0.0002, cross_up(True) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 16:56 (bo) : HOLD high 0.82653, low 0.82222, sar 0.8124, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0032, macdhist 0.0004, cross_up(True) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 17:01 (bo) : HOLD high 0.82653, low 0.82348, sar 0.8127, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0033, macdhist 0.0004, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 17:06 (bo) : HOLD high 0.82782, low 0.82355, sar 0.813, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0033, macdhist 0.0003, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 17:11 (bo) : HOLD high 0.82466, low 0.82036, sar 0.8136, shift_up(

Aug 19 19:50 (bo) : HOLD high 0.84161, low 0.83657, sar 0.8199, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0058, macdhist 0.0022, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 19:55 (bo) : HOLD high 0.84284, low 0.83887, sar 0.8221, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0062, macdhist 0.0021, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 20:00 (bo) : HOLD high 0.84273, low 0.83924, sar 0.8246, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0066, macdhist 0.0019, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 20:05 (bo) : HOLD high 0.8484, low 0.84302, sar 0.8268, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0071, macdhist 0.002, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 19 20:10 (bo) : HOLD high 0.8496, low 0.84324, sar 0.8324, shift_u

Aug 20 01:06 (bo) : HOLD high 0.87295, low 0.86359, sar 0.8535, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0042, macdhist 0.001, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 01:11 (bo) : HOLD high 0.87024, low 0.86, sar 0.855, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0039, macdhist 0.0006, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 01:16 (bo) : HOLD high 0.86638, low 0.86243, sar 0.8565, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0039, macdhist 0.0004, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 01:21 (bo) : HOLD high 0.8681, low 0.86246, sar 0.8578, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.004, macdhist 0.0004, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 01:26 (bo) : HOLD high 0.87135, low 0.86532, sar 0.859, shift_up(Fal

Aug 20 04:05 (so) : HOLD high 0.84933, low 0.84675, sar 0.8537, shift_up(False) shift_down(False) below_low(False) above_high(True), macd -0.0021, macdhist -0.0004, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 20 04:11 (so) : HOLD high 0.8492, low 0.84555, sar 0.8529, shift_up(False) shift_down(False) below_low(False) above_high(True), macd -0.0025, macdhist -0.0006, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 20 04:15 (so) : HOLD high 0.84705, low 0.84273, sar 0.8519, shift_up(False) shift_down(False) below_low(False) above_high(True), macd -0.0028, macdhist -0.0007, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 20 04:21 (so) : HOLD high 0.847, low 0.84343, sar 0.8504, shift_up(False) shift_down(False) below_low(False) above_high(True), macd -0.0031, macdhist -0.0008, cross_up(False) cross_down(False) below_zero(True) above_zero(False) 
Aug 20 04:26 (so) : HOLD high 0.84436, low 0.84117, sar 0.8492,

Aug 20 07:05 (bo) : HOLD high 0.8533, low 0.853, sar 0.8501, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0013, macdhist 0.0004, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 07:11 (bo) : HOLD high 0.85564, low 0.85303, sar 0.8506, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0015, macdhist 0.0004, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 07:16 (bo) : HOLD high 0.85939, low 0.85459, sar 0.851, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.0019, macdhist 0.0007, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 07:21 (bo) : HOLD high 0.85665, low 0.85233, sar 0.8516, shift_up(False) shift_down(False) below_low(True) above_high(False), macd 0.002, macdhist 0.0006, cross_up(False) cross_down(False) below_zero(False) above_zero(True) 
Aug 20 07:26 (bo) : HOLD high 0.85656, low 0.85429, sar 0.8523, shift_up(